In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

In [ ]:
box_DIR = "../data/box/"

In [ ]:
year = 2023
cols = ['gameId', 'teamId', 'teamName',"personId","firstName","familyName",'minutes']
# df0 = pd.read_parquet(box_DIR +f"NBA_Box_P_Adv_{year}"+".parquet")
df0 = pd.read_parquet(box_DIR +f"NBA_Box_P_Adv_{year}"+".parquet",columns=cols)
df0["playerName"] = df0["firstName"] + " " + df0["familyName"]
df0["minutes"] = df0["minutes"].replace("","00:00")
df0[["mins","secs"]] = df0["minutes"].str.split(":",expand=True)
df0["mins"] = df0["mins"].astype(int)
df0["secs"] = df0["secs"].astype(int)

In [ ]:
cols = ["GAME_ID","TEAM_ID"]
df1 = pd.read_parquet(box_DIR +f"NBA_Box_T_Base_{year}"+".parquet",columns=cols)
df1 = df1.rename(columns={"GAME_ID":"gameId","TEAM_ID":"teamId"})
df1["gameId"] = df1["gameId"].astype(int)

In [ ]:
df2 = pd.merge(df0,df1,on=["gameId","teamId"])

In [ ]:
df2.query('familyName ==  "Quickley"')

In [ ]:
df31 = pd.DataFrame(df2.query("mins >=20").groupby(["personId","playerName"])["gameId"].count())
df31 = df31.reset_index()
df31 = df31.rename(columns={"gameId":"M20"})

In [ ]:
df32 = pd.DataFrame(df2.query("mins >=15").query("mins <20").groupby(["personId","playerName"])["gameId"].count())
df32 = df32.reset_index()
df32 = df32.rename(columns={"gameId":"B1520"})

In [ ]:
df3 = pd.merge(df31,df32)

In [ ]:
cols = ["PLAYER_ID","TEAM_ID","PLAYER_NAME","GP"]
df4 = pd.read_parquet(box_DIR +f"NBA_Box_P_Cum_Base_{year}"+".parquet")
# df4 = pd.read_parquet(box_DIR +f"NBA_Box_P_Cum_Base_{year}"+".parquet",columns=cols)
df4 = df4.rename(columns={"PLAYER_ID":"personId","TEAM_ID":"teamId"})

In [ ]:
df41 = df4.sort_values('PLAYER_NAME')

In [ ]:
df5 = pd.read_parquet(box_DIR +f"NBA_Box_T_Base_{year}"+".parquet")

In [ ]:
season = str(year) + "-" + str(year + 1)[-2:]
stats = leaguedashteamstats.LeagueDashTeamStats(per_mode_detailed="PerGame", season_type_all_star="Regular Season", season = season)
df5 = stats.get_data_frames()[0]
df5 = df5.rename(columns={"TEAM_ID":"teamId"})
df5 = df5[["teamId","GP"]]

In [ ]:
df6 = pd.merge(df4,df5,on=["teamId"])

In [ ]:
df6["GP_D"] = df6["GP_y"] - df6["GP_x"]

In [ ]:
df6.query('PLAYER_NAME ==  "RJ Barrett"')

In [ ]:
df7 = pd.merge(df3,df6,on=)